# Graph AutoEncoder with StellarGraph

In [1]:
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding


from tensorflow import keras
from stellargraph import datasets

2024-03-26 22:24:10.883232: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-26 22:24:10.883257: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-26 22:24:12.512415: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-03-26 22:24:12.512592: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-26 22:24:12.512603: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-26 22:24:12.512618: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [2]:
dataset = datasets.Cora()
G, _ = dataset.load()

In [3]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)
        Features: none


In [4]:
edge_splitter_test = EdgeSplitter(G)

G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

edge_splitter_train = EdgeSplitter(G_test)

G_train, edge_ids_train, edge_labels_train = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 542 positive and 542 negative edges. **
** Sampled 542 positive and 542 negative edges. **


In [5]:
train_gen = FullBatchLinkGenerator(G, method="gcn")
train_flow = train_gen.flow(edge_ids_train, edge_labels_train)

Using GCN (local pooling) filters...


In [6]:
test_gen = FullBatchLinkGenerator(G, method="gcn")
test_flow = train_gen.flow(edge_ids_test, edge_labels_test)

Using GCN (local pooling) filters...


In [7]:
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)

In [8]:
x_inp, x_out = gcn.in_out_tensors()

In [9]:
x_inp

[<KerasTensor: shape=(1, 2708, 1433) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(1, None, 2) dtype=int32 (created by layer 'input_2')>,
 <KerasTensor: shape=(1, None, 2) dtype=int64 (created by layer 'input_3')>,
 <KerasTensor: shape=(1, None) dtype=float32 (created by layer 'input_4')>]

In [10]:
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

In [11]:
prediction = keras.layers.Reshape((-1,))(prediction)


In [12]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.Accuracy()],
)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(1, 2708, 1433)]    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(1, None, 2)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(1, None)]          0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (1, 2708, 1433)      0           input_1[0][0]                    
______________________________________________________________________________________________

In [14]:
history = model.fit(train_flow, validation_data=test_flow, epochs=50)

2024-03-26 22:24:31.547943: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-03-26 22:24:31.548360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2803200000 Hz


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 1.6882 - accuracy: 0.0000e+00 - val_loss: 1.7383 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 141ms/step - loss: 1.8908 - accuracy: 0.0000e+00 - val_loss: 0.5827 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 119ms/step - loss: 0.6209 - accuracy: 0.0000e+00 - val_loss: 0.6141 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 133ms/step - loss: 0.6203 - accuracy: 0.0000e+00 - val_loss: 0.5935 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 118ms/step - loss: 0.5852 - accuracy: 0.0000e+00 - val_loss: 0.5622 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 143ms/step - loss: 0.5834 - accuracy: 0.0000e+00 - val_loss: 0.6076 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 151ms/step - loss: 0.6186 - accuracy: 0.0000e+00 - val_loss: 0